# Calculate and graph performance summary statistics #

** Author: Andrew Larkin **, Oregon State University College of Public Health and Human Sciences <br>
** Date created: ** November 28, 2018

### Summary ###
Summary statistics image and remote sensing measures at PlacePulse image locations.  Operations include calculating summary statistics, plotting histograms and boxplots, and creating correlation matrices and correlation plots.

### setup ###

In [ ]:
import pandas as ps
import csv
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns; sns.set(color_codes=True) # note: do not import seaborn if you want outliers to show in boxplots

In [ ]:
# define filepaths
parentFolder = "C:/users/larkinan/desktop/PlacePulse/"#"G:/dropbox/PlacePulse/"
measuresCSV = parentFolder + "PlacePulseMergedMeasures_Dec13p3_18.csv"
addedConstructsCSV = parentFolder + "PlacePulseMergedMeasures_Dec13p4_18.csv"


correlationVarsReduced = ['accessibility','allNature','animate','bluespace','building','builtEnv','car','grass','greenspace',
                          'otherNature','person','plant','road','sidewalk','sky','tree','bench',
                          'PM251000m','mjRds100m','NO2100m','tr100m','imp1000m','pop1000m','NDVI250m',
                          'mu_beautiful','mu_lively','mu_safety']

# variables to include in correlation matrix and correlation plot
correlationVars = ['accessibility','allNature','animate','bluespace','building','builtEnv','car','grass','greenspace',
                   'otherNature','person','plant','road','sidewalk','sky','tree','bench',
                   'PM251000m','mjRds100m','mjRds250m','NO2100m','NO2250m','tr100m','tr250m','imp1000m','pop1000m',
                   'NDVI250m',
                   'mu_beautiful','mu_lively','mu_safety']
    


### calculate statistics for one level in a stratificaiton ###
function does not return statistics, but rather appends to already existing variable arrays.  One array corresponds to a single column in a denormalized database.
**Inputs** <br>
- **inputDataset** (pandas dataframe) - contains a large set of variables, including all variables of interest <br>
- **stratifyVal** (string) - stratification level value <br>
- **valueNames** (string array) - variables to calculate summary statistics for <br>
- **stratifyName** (string array) - array designiating the stratify value for an index in all arrays <br>
- **value** (string array) - array designating the variable category for an index in all arrays <br>
- **meanVals** (float array) - mean value for the variable in value and stratification in stratifyNames <br>
- **stdDev** (float array) - std dev for the variable in value and stratification in stratifyNames <br>
- **minVal** (float aray) - min value for the variable in value and stratification in stratifyNames <br>
- **maxVal** (float array) - max value for the variable in value and stratifcation in stratifyNames <br>
- **iqr** (float array) interquartile range for the variable in value and stratification in stratifyNames <br>

In [ ]:
def calcStatsOneStratify(inputDataset,stratifyVal,valueNames,
                        stratifyName,value,meanVals,stdDev,minVal,maxVal,iqr):
    
    # for each varaible of interest, calculate summary statistics and append results to arrays
    for valueName in valueNames:
        stratifyName.append(stratifyVal)
        value.append(valueName)
        
        npArr = np.array(inputDataset[valueName])
        npArr = npArr[np.logical_not(np.isnan(npArr))]
        meanVals.append(np.mean(npArr))
        stdDev.append(np.std(npArr))
        q75, q25 = np.percentile(npArr, [75 ,25])
        minVal.append(np.min(npArr))
        maxVal.append(np.max(npArr))
        iqr.append(q75 - q25)

### calculate statistics, stratified by a specific variable ###
**Inputs** <br>
- **inputDataset** (pandas dataframe) - contains all variables and records of interest <br>
- **valueNames** (string array) - names of the variables to calculate summary statistics for <br>
- **stratify** (string) - name of the variable to stratify by <br>

**Outputs** <br>
- **df (pandas dataframe)** - contains derived summary statistics <br>

In [ ]:
def calcSummaryStats(inputDataset,valueNames,stratify):
    
    # get set of all stratification levels
    stratifyLevels = set(inputDataset[stratify])
    stratifyName, value, meanVals, stdDev, minVal, maxVal, iqr  = ([] for i in range(7))
    
    # for each stratifcation level, get the subset partition of data 
    # and caculate summary statistics for the partition
    for stratifyLevel in stratifyLevels:
        subsetData = inputDataset.loc[inputDataset[stratify] == stratifyLevel]
        calcStatsOneStratify(subsetData,stratifyLevel,valueNames,
                            stratifyName,value,meanVals,stdDev,minVal,maxVal,iqr)
        
    # calculate summary statistics for the entire dataset, without stratification
    calcStatsOneStratify(inputDataset,"none",valueNames,
                        stratifyName,value,meanVals,stdDev,minVal,maxVal,iqr)
    
    # combine summary statistic arrays into a dict
    statsDict = {stratify:stratifyName,'meanVals':meanVals,'stdDev':stdDev,'IQR':iqr,'value':value,
                'min':minVal,'max':maxVal}
    df = ps.DataFrame.from_dict(statsDict)
    return(df)

### calculate 25th and 75th percentile for a subset of variables of interest ###
**Inputs** <br>
- **inputDataset** (pandas dataframe) - contains all variables of interest <br>
- **categories** (string array) - names of the variables to calculate percentiles for <br>

**Outputs** <br>
- **outputDict** (dict) - dictionary containing 25th and 75th percentile for each variable of interest <br>

In [ ]:
def calcLowHighQuartiles(inputDataset,categories):
    outputDict = {}
    
    # for each variable of interest
    for category in categories:
        dataSubset = inputDataset[category]
        dataSubset = dataSubset.dropna()
        outputDict[category + "low"] = np.percentile(dataSubset, 25)
        outputDict[category + "high"] = np.percentile(dataSubset,75)
    return(outputDict)

### calculate summary statistics for a specific subset of variables of interest.  For completing descriptive statistics in manuscript ###
**Inputs** <br>
- **inputDataset** (pandas dataframe) - contains all variables and records of interest <br>
- **outputFilename** (string) - output filepath to write results to <br.

In [ ]:
def subsetStatsForTable(inputDataset,outputFilename):
    
    # variables to calculate summary statistics for 
    tableCategories = ['accessibility','allNature','animate','bluespace','building','builtEnv','car','grass','greenspace',
                       'otherNature','person','plant','road','sidewalk','sky','tree','person','bench',
                       'PM251000m','mjRds100m','mjRds250m','NO2100m','NO2250m','tr100m','tr250m','imp1000m','pop1000m',
                       'NDVI250m',
                       'mu_beautiful','mu_lively','mu_safety','count_beautiful','win_beautiful','lose_beautiful',
                       'count_lively','win_lively','lose_lively','count_safety','win_safety','lose_safety',
                       'CITY_NAME']
    
    # subset dataset and calculate summary statistics
    tableSubset = inputDataset[tableCategories]
    tableCategories = tableCategories[0:len(tableCategories)-1]
    subsetStats = calcSummaryStats(tableSubset,tableCategories,'CITY_NAME')
    subsetStats.to_csv(outputFilename + "_" + str(len(tableSubset['CITY_NAME'])) + ".csv")                 

### calculate summary statistics for the bottom and top quartile of perception variables ###
**Inputs** <br>
- **inputDataset** (pandas dataframe) - contains all variables and records of interest <br>
- **parentFolder** (string) - name of the folder to write results to <br>
- **valueNames** (string array) - names of the variables to calculate summary statistics for <br>
- **stratify** (string) - name of the variable to stratify by

In [ ]:
def calcLowHighSummaryStats(inputDataset,parentFolder,valueNames):
    
    # perception variables to calculate bottom and top percentile and partition by 
    percentileVars = ['mu_beautiful','mu_lively','mu_safety']
    
    # for each perception variable, calculate bottom and top quartile
    lowHighDict = calcLowHighQuartiles(inputDataset,percentileVars)
    
    # for each perception variable, subset the bottom and top quartiles and calculate summary stats
    for percentileVar in percentileVars:
            lowSubset = inputDataset.loc[inputDataset[percentileVar] < lowHighDict[percentileVar + "low"]]
            subsetStatsForTable(lowSubset,parentFolder + percentileVar + "low")
            highSubset = inputDataset.loc[inputDataset[percentileVar] > lowHighDict[percentileVar + "high"]]
            subsetStatsForTable(highSubset,parentFolder + percentileVar + "high")
    print(lowHighDict)

### create a histogram for a single variable of interest.  Part of a larger plot of multiple variables ###
**Inputs** <br>
- **xMax** (int) - max value for the x axis <br>
- **xVals** (float array) - values used to create histograms <br>
- **subplotIndex** (int) - index of the larger plot where the subplot will be placed <br>
- **yLabel** (string) - name of the y axis <br>
- **xLabel** (string) - name of the x axis <br>

In [ ]:
def createHistogram(xMax,xVals,subplotIndex,yLabel,xLabel):
    tempAxis = plt.subplot(5,2,subplotIndex)
    tempAxis.set_xlim([min(xVals),xMax])
    plt.xlabel(xLabel)
    plt.ylabel(yLabel)
    n, bins, patches = plt.hist(xVals, 50,facecolor='g', alpha=0.75)

### plot historgrams for multiple variables, stratified by a specific variable ### 
**Inputs** <br>
- **inData** (pandas dataframe) - contains all variables and records of interest <br>
- **outFolder** (string) - where histogram plots are saved <br>
- **stratify** (string) - name of the variable containing stratification levels <br>
- **valuesOfInterest** (string array) - name of the variables to create histograms for <br>

In [ ]:
def createHistograms(inData,outFolder,stratify,valuesOfInterest):
    
    # names of the stratification levels
    stratifyLevels = set(inData[stratify])
    
    # for each stratification level, subset data by stratificatoin, and create a plot to store histograms
    for stratifyLevel in stratifyLevels:
        outputFile = outFolder + "RemSensHist_" + str(stratifyLevel) + ".png"
        if not(os.path.exists(outputFile)):
            stratifySubset = inData.loc[inData[stratify] == stratifyLevel]
            fig = plt.figure(num=None, figsize=(12, 15), dpi=160, facecolor='w', edgecolor='k')
            fig.suptitle("Remote Sensing " + stratifyLevel, fontsize=16)
            index = 1
            
            # for each value of interest, create a histogram subplot
            for value in valuesOfInterest:
                subsetData = stratifySubset[value]
                createHistogram(max(subsetData),subsetData,index,"frequency",value)
                index+=1
            plt.savefig(outFolder + "RemSensHist_" + str(stratifyLevel) + ".png", bbox_inches="tight")
            
    # create histograms for all data without stratification 
    stratifySubset = inData
    fig = plt.figure(num=None, figsize=(12, 15), dpi=160, facecolor='w', edgecolor='k')
    fig.suptitle("Remote Sensing " + "AllLevels", fontsize=16)
    index = 1
    for value in valuesOfInterest:
        subsetData = stratifySubset[value]
        createHistogram(max(subsetData),subsetData,index,"frequency",value)
        index+=1
        plt.savefig(outFolder + "RemSensHist_All.png", bbox_inches="tight")
    
    
    #plt.show() # only use for debugging purposes

### createBoxplots for multiple variables, stratified by a specific variable ### 
**Inputs** <br>
- **inData** (pandas dataframe) - contains all variables in the records of interest <br>
- **outFolder** (string) - where boxplots are saved <br>
- **stratify** (string) - name of the variable containing stratification levels <br>
- **valuesOfInterest** (string array) - names of the variables to create boxplots for <br>

In [ ]:
def createBoxplots(inData,outFolder,stratify,valuesOfInterest):
    
    # names of the stratification levels
    stratifyLevels = set(inData[stratify])
    
    # for each variable of interest, calculate summary statistics for each stratification level 
    for value in valuesOfInterest:
        outputFile = outFolder + "MeasBoxPlot_" + value + ".png"
        if not os.path.exists(outputFile):
            fig = plt.figure(num=None, figsize=(48, 15), dpi=160, facecolor='w', edgecolor='k')
            fig.suptitle("Measure " + value, fontsize=16)
            boxData, medianVals, cityNames, cityNamesSorted, boxDataSorted  = ([] for i in range(5))
            index=0
        
            for stratifyLevel in stratifyLevels:
                stratifySubset = inData.loc[inData[stratify] == stratifyLevel]
                stratifySubset = stratifySubset.dropna()
                stratifySubset = stratifySubset[value]
                boxData.append(stratifySubset)
                medianVals.append((index,np.median(stratifySubset)))
                cityNames.append(stratifyLevel)
                index+=1
                
            # order the stratified summary statistics from low to high median
            medianVals.sort(key=lambda x:x[1])
            
            for sortedMedian in medianVals:
                boxDataSorted.append(boxData[sortedMedian[0]])
                cityNamesSorted.append(cityNames[sortedMedian[0]])
            plt.boxplot(boxDataSorted)
            plt.xticks(range(1,len(cityNamesSorted)+1),cityNamesSorted,rotation=70)
            #plt.show()
            plt.savefig(outFolder + "MeasBoxPlot_" + value + ".png")
            plt.close()

### sum multiple variables to create latent constructs ###
**Inputs** <br>
- **inData** (pandas dataframe) - contains all variables and records of interest <br>
- **categories** (string array) - names of the variables to sum <br>
- **categoryName** (string) - name to designate for the latent construct <br>

In [ ]:
def createCategory(inData,categories,categoryName):
    tempData = np.zeros((len(inData['wall']),1),dtype=float)
    index=0
    for category in categories:
        tempData = np.add(tempData,np.array(inData[category],dtype=float).reshape(len(inData['wall']),1))
    inData[categoryName] = tempData

### create latent constructs by summing variables ###
Adds latent constructs in place, nothing is returned from the function <br>
**Inputs**<br>
- **InData** (pandas dataframe) - contains all variables and records of interest <br>

In [ ]:
def createCategories(inData):

    # crate the built environment category
    builtEnv = ['wall','building','road','windowpane','sidewalk','hovel','house','fence','railing',
               'signboard','skyscraper','path','stairs','runway','screen', 'door', 'screen door','stairway',
                'bridge','bench','booth','awning','streetlight','pole','bannister','escalator',
               'fountain','swimming pool','step','sculpture','traffic light','pier','bulletin board']
    createCategory(inData,builtEnv,'builtEnv')
    
    # create the accessibility category 
    accessibility = ['sidewalk','escalator','path','stairs','stairway','bench','step']
    createCategory(inData,accessibility,'accessibility')
    
    # create the allNature category
    allNature = ['tree','grass','plant','field','land','flower','water','sea','waterfall','lake','earth',
                'mountain','rock','sky','sand','hill','dirt track']
    createCategory(inData,allNature,'allNature')
    
    # create the greenspace cateogry 
    greenspace = ['tree','grass','plant','field','flower']
    createCategory(inData,greenspace,'greenspace')
    
    # create the bluespace category 
    bluespace = ['water','sea','waterfall','lake']
    createCategory(inData,bluespace,'bluespace')
    
    # create the otherNature category
    otherNature = ['earth','mountain','rock','sky','sand','hill','dirt track','land']
    createCategory(inData,otherNature,'otherNature')
    
    # create the animate category
    animate = ['person','boat','car','bus','truck','airplane','van','ship','minibike','animal','bicycle']
    createCategory(inData,animate,'animate')
    
    indoors = ['ceiling','chair','table','painting','sofa','shelf','desk','chest of drawers','counter',
              'case','pool table','pillow','bookcase','coffee table','countertop','toilet',
               'stove','kitchen island','swivel chair','chandelier','ottoman','buffet',
              'cradle','microwave','dishwasher','plate','monitor','clock','rug']
    
    createCategory(inData,indoors,'indoors')
    

### calculate correlations and create a correlation plot for the entire dataset ###
**Inputs** <br>
- **outFolder** (string) - name of the folder to store correlation matrix and correlation plot

In [ ]:
def createCorrPlot(inData,outFolder):

    correlationSubset = inData[correlationVarsReduced]
    
    # calculate correlations 
    corr = correlationSubset.corr()
    fig = plt.figure(num=None, figsize=(48, 15), dpi=160, facecolor='w', edgecolor='k')
    
    # create correlation plot
    sns.clustermap(corr,cmap="coolwarm_r")#, robust=True)
    #plt.show()
    plt.savefig(outFolder + "CorrPlot.eps")
    plt.close()
    #print(corr)
    corr.to_csv(outFolder + "corrMatrix.csv")

### reorder a set of observations based on the order of a separate array and normalize output ###

**Inputs** <br>
- **inDataset** (pandas dataframe) - contains all variables of interest <br>
- **citySortOrder** (string array) - array of city names in order to sort dataset <br>
- **attribute** (string) - name of the attribute to sort <br>

**Outputs** <br>
- **graphDict** (dict) - normalized mean and standard deviation of the attribute of interest, ordered by city array <br>


In [1]:
def setSortOrder(inDataset, citySortOrder,attribute):
    
    stdDev, meanVals = ([] for i in range(2))
    
    # sort mean and standard deviation of variable of interst based on city name
    for cityName in citySortOrder:
        meanVals.append(float(inDataset[inDataset.CITY_NAME == cityName]['meanVals']))
        stdDev.append(float(inDataset[inDataset.CITY_NAME == cityName]['stdDev']))
    
    # normalize mean and std dev of attribute
    normVals = np.divide(np.subtract(meanVals,np.mean(meanVals)),np.std(meanVals))
    stdDev = np.divide(stdDev,stdDev)
    
    graphDict = {'mean':normVals,'stdDev':stdDev,'city':citySortOrder}
    return(graphDict)

### calculate normalized mean and standard deviation of city level trends and plot in sorted order ###
**Inputs** <br>
- **attributes** (string array) - names of the variables to calc summary stats and plot.  The first string desginates the sort order of the plot.  The first attribute used to sort is not plotted <br>
- **title** (string) - title to put on the graph <br>
- **yLim** (float array) - min and max y limits <br>
- **outputFilename** (string) - output filename of the graph.  Relative rather than absolute filepath. <br>

In [ ]:
def plotCityLevelTrends(attributes,title,yLim,outputFilename):
    
    # load city level estimates from csv
    estimates = ps.read_csv(parentFolder + "remoteSensingsummaryStats.csv")
    
    graphDictArray, sortOrder, cityOrder, graphDict  = ([] for i in range(4))

    for attribute in attributes:
        attributeSubset = estimates[estimates.value == attribute]

        # use the mean of the first attribute to determine the order of all attributes plotted in the graph
        if(len(sortOrder) == 0):
            sortOrder = attributeSubset.sort_values('meanVals')
            cityOrder = list(sortOrder['CITY_NAME'])
        else:
            graphDictArray.append(setSortOrder(attributeSubset,cityOrder,attribute))
                        
    fig = figure(num=None, figsize=(14, 10), dpi=160, facecolor='w', edgecolor='k')
    fig.suptitle(title, fontsize=16)
    
    colorVec = ['#90ee90','#2f4b7c','#ffa500','#a05195','#d45087','#f95d6a','#ff7c43','#ffa600']
    xVals = range(0,len(graphDictArray[0]['city']))
    
    numCities = len(xVals)
    stepLen = numCities
    numParams = len(graphDictArray)
    
    plt.xticks(np.arange(0, numCities + 1, step=stepLen))
    
    # for each attribute in the graph, plot the points and std dev bars
    for i in range(0,len(attributes)-1):
        currYVals = np.asarray(graphDictArray[i]['mean'])
        currYVals = currYVals.reshape((len(xVals),))
        yErrVals = np.asarray(graphDictArray[i]['stdDev']).reshape((len(xVals),))
        currColor = colorVec[i]
        plt.subplot()
        plt.scatter( xVals, currYVals,label = attributes[i+1], alpha = 1.0, color=currColor)
        plt.errorbar(xVals, currYVals, yerr=yErrVals, fmt='o',alpha = 0.4,color = currColor)
        
    plt.gca().set_ylim(yLim)
    plt.legend()
    plt.savefig(parentFolder + outputFilename, bbox_inches="tight")
    plt.close()

### calculate correlation between cities and graph the results in a correlation plot.  ###
**Inputs** <br>
- **attributes** (string array) - names of variables to include in the correlation matrix <br>
- **outFolder** (string) - folderpath where the correlation matrix and plot will be stored <br>

In [ ]:
def calcCityLevelCorr(attributes,outFolder):
    # load city level estimates from csv
    estimates = ps.read_csv(parentFolder + "remoteSensingsummaryStats.csv")
    graphDictArray = []
    
    cityNames = list(set(estimates['CITY_NAME']))
    cityDataset = ps.DataFrame(cityNames)
    cityDataset.columns = (['CITY_NAME'])
    cityDataset = cityDataset.sort_values('CITY_NAME')
    
    for attribute in attributes:
        attributeSubset = estimates[estimates.value == attribute]
        attributeSubset = attributeSubset.sort_values('CITY_NAME')
        attributeVals = list(attributeSubset['meanVals'])
        cityDataset[attribute] = attributeVals
    

    
    correlationSubset = cityDataset.drop(['CITY_NAME'],axis=1)
    
    # calculate correlations 
    corr = correlationSubset.corr()
    fig = plt.figure(num=None, figsize=(48, 15), dpi=160, facecolor='w', edgecolor='k')
    
    # create correlation plot
    sns.clustermap(corr,cmap="coolwarm_r")#, robust=True)
    #plt.show()
    plt.savefig(outFolder + "CorrPlotCityLevel.eps")
    plt.close()
    #print(corr)
    corr.to_csv(outFolder + "corrMatrixCity.csv")
    

### calculate average correlation within cities and graph the results in a correlation plot.  City averages are weighted by number of records ###
**Inputs** <br>
- **inData** (pandas dataframe) - dataset containing all variables of interest and city names <br>
- **outFolder** (string) - name of the folder where the correlation plot will be stored <br>

In [2]:
def calcIntraCityCorr(inData,outFolder):
    
    cityNames = list(set(inData['CITY_NAME']))
    cumulativeCorrSum = np.zeros((len(correlationVarsReduced),len(correlationVarsReduced)))

    for city in cityNames:
        
        cityDataset = inData[inData.CITY_NAME == city]
        cityDataset = cityDataset[correlationVarsReduced]
        corr = cityDataset.corr()
        
        numCitySamples = len(cityDataset['animate'])
        numVars = len(correlationVarsReduced)
        
        # for one city, there is no change in tr100 and, consequently NA correlation values.  This is to properly 
        # weight these values
        numZeroCorrValues = np.isnan(corr)*numCitySamples

        corr = np.nan_to_num(corr)
        corrWeights = np.full((numVars,numVars),numCitySamples)
        corrWegihts = np.subtract(corrWeights,numZeroCorrValues)
        corr = np.multiply(corr,corrWeights)       
        cumulativeCorrSum = np.add(corr,cumulativeCorrSum)

    #print(len(inData['animate']))
    avgCor = np.divide(cumulativeCorrSum,len(inData['animate']))
    
    #print(avgCor)
    fig = plt.figure(num=None, figsize=(48, 15), dpi=160, facecolor='w', edgecolor='k')
    
    # create correlation plot
    sns.clustermap(avgCor,cmap="coolwarm_r")#, robust=True)
    #plt.show()
    plt.savefig(outFolder + "CorrPlotIntraCity2.eps")
    plt.close()
    

### main function ###

In [ ]:
def main():
    
    # load data and clean
    measuresData = ps.read_csv(measuresCSV)
    removeVals = ['id','latit','longit','latJoin','longJoin','latitude','longitude','CITY_NAME',
             'GMI_ADMIN','ADMIN_NAME','FIPS_CNTRY','CNTRY_NAME','STATUS','POP','POP_RANK','POP_CLASS']
    
    
    screenedData = measuresData.dropna(subset=['wall']) 
    
    # create latent constructs 
    createCategories(screenedData)
    

    valuesOfInterest = ([val for val in list(screenedData) if val not in removeVals])
    
    screenedData.to_csv(addedConstructsCSV)
    
    screenedData = screenedData[screenedData.indoors < 0.1]
    print("numEntries after removing indoor outliers: %i", len(screenedData['wall']))
    
    calculate summary stats
    calcLowHighSummaryStats(screenedData,parentFolder,valuesOfInterest)
    allSumStats = calcSummaryStats(screenedData,valuesOfInterest,'CITY_NAME')
    allSumStats.to_csv(parentFolder + "remoteSensingsummaryStats.csv")
    
    create histograms, boxplots and correlation plots
    #createHistograms(screenedData,parentFolder,'CITY_NAME',valuesOfInterest)
    createBoxplots(screenedData,parentFolder,'CITY_NAME',valuesOfInterest)
    createCorrPlot(screenedData,parentFolder)
    calcCityLevelCorr(correlationVarsReduced,parentFolder)
    calcIntraCityCorr(screenedData,parentFolder)
      
    plotCityLevelTrends(['mu_beautiful','mu_safety','mu_beautiful','mu_lively'],'construct scores',[-7.5,7.5],'constructScores.eps')
    plotCityLevelTrends(['mu_beautiful','greenspace','NDVI250m','mu_beautiful'],'nature scores',[-7.5,7.5],'natureScores.eps')
    plotCityLevelTrends(['mu_beautiful','road','mu_beautiful','mjRds100m','builtEnv'],'built env scores',[-7.5,7.5],'builtEnvScores.eps')

In [ ]:
main()